In [115]:
import numpy as np
import pandas as pd
import os
import sklearn as sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Reading Data

In [31]:
def segmentWords(s): 
    return s.split()

def readFile(fileName):
    # Function for reading file
    # input: filename as string
    # output: contents of file as list containing single words
    contents = []
    f = open(fileName)
    for line in f:
        contents.append(line)
    f.close()
    result = segmentWords('\n'.join(contents))
    return result

In [32]:
for c in os.listdir("data_training"):
    directory = "data_training/" + c
    for file in os.listdir(directory):
        print(file)
        words = readFile(directory + "/" + file)
        print(len(words))
        break

cv000_29416.txt
825
cv000_29590.txt
802


In [70]:
os.listdir("data_training")

['neg', 'pos']

#### Create a Dataframe containing the counts of each word in a file

In [80]:
d = []

for c in os.listdir("data_training"):
    directory = "data_training/" + c
    for file in os.listdir(directory):
        words = readFile(directory + "/" + file)
        e = {x:words.count(x) for x in words}
        #e['__FileID__'] = file
        e['__CLASS__'] = c
        d.append(e)

Create a dataframe from d - make sure to fill all the nan values with zeros.

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html


In [81]:
df = pd.DataFrame(d)
dataframe = df.fillna(0)

#### Split data into training and validation set 

* Sample 80% of your dataframe to be the training data

* Let the remaining 20% be the validation data (you can filter out the indicies of the original dataframe that weren't selected for the training data)

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [91]:
training_data = dataframe.sample(frac = 0.8)#.drop(labels = ['__FileID__'],axis = 1)
validation_data = dataframe.drop(labels =training_data.index, axis = 0)#.drop(labels = ['__CLASS__'],axis = 1)


#training_data,validation_data = train_test_split(dataframe.values, test_size=0.2)

In [93]:
training_data['__CLASS__']

227     neg
762     neg
934     pos
1215    pos
1341    pos
641     neg
1268    pos
316     neg
1522    pos
785     neg
849     pos
125     neg
26      neg
1076    pos
1086    pos
1547    pos
1241    pos
1576    pos
233     neg
1276    pos
1503    pos
104     neg
1176    pos
1373    pos
153     neg
1195    pos
1425    pos
1071    pos
649     neg
305     neg
       ... 
783     neg
1275    pos
1468    pos
1448    pos
832     pos
1287    pos
580     neg
1593    pos
1599    pos
1552    pos
528     neg
308     neg
1119    pos
527     neg
570     neg
1043    pos
588     neg
1529    pos
666     neg
189     neg
681     neg
1078    pos
543     neg
1334    pos
365     neg
67      neg
1147    pos
164     neg
94      neg
1379    pos
Name: __CLASS__, Length: 1280, dtype: object

* Split the dataframe for both training and validation data into x and y dataframes - where y contains the labels and x contains the words

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [95]:
x_training_data = training_data.drop(labels = '__CLASS__',axis = 1)
y_training_data = training_data['__CLASS__']
x_validation_data = validation_data.drop(labels = '__CLASS__',axis = 1)
y_validation_data = validation_data['__CLASS__']

# Logistic Regression

#### Basic Logistic Regression
* Use sklearn's linear_model.LogisticRegression() to create your model.
* Fit the data and labels with your model.
* Score your model with the same data and labels.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [104]:
#using default regularization term
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
my_logistic_reg = logistic.fit(x_training_data,y_training_data)
print('Score of Logistic Regression using default regularization term: ',my_logistic_reg.score(x_validation_data,y_validation_data))

Score of Logistic Regression using default regularization term:  0.81875


I am not overfitting, a score of 0.8187 is really good

#### Changing Parameters

In [109]:
#using customized regularization term and methodology
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(C = 1e6)
my_logistic_reg = logistic.fit(x_training_data,y_training_data)
my_logistic_reg.score(x_validation_data,y_validation_data)
print('Score of Logistic Regression using customized regularization term and newton-cg method: ',my_logistic_reg.score(x_validation_data,y_validation_data))


Score of Logistic Regression using customized regularization term and newton-cg method:  0.796875


#### Feature Selection
* In the backward stepsize selection method, you can remove coefficients and the corresponding x columns, where the coefficient is more than a particular amount away from the mean - you can choose how far from the mean is reasonable.

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.std.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.mean.html

How did you select which features to remove? Why did that reduce overfitting?

# Single Decision Tree

#### Basic Decision Tree

* Initialize your model as a decision tree with sklearn.
* Fit the data and labels to the model.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html


In [120]:
clf = DecisionTreeClassifier(random_state=0)
decision_tree = clf.fit(x_training_data,y_training_data)

In [126]:
decision_tree.tree_

#### Changing Parameters
* To test out which value is optimal for a particular parameter, you can either loop through various values or look into sklearn.model_selection.GridSearchCV

References:


http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

How did you choose which parameters to change and what value to give to them? Feel free to show a plot.

Why is a single decision tree so prone to overfitting?

# Random Forest Classifier

#### Basic Random Forest

* Use sklearn's ensemble.RandomForestClassifier() to create your model.
* Fit the data and labels with your model.
* Score your model with the same data and labels.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


#### Changing Parameters

What parameters did you choose to change and why?

How does a random forest classifier prevent overfitting better than a single decision tree?